In [1]:
import pubchempy as pcp
import numpy as np
import py3Dmol
import os
from io import StringIO
from Bio.PDB import PDBParser, PDBList
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider, Text, Dropdown, ToggleButton, Button, FloatSlider, Checkbox, SelectMultiple
from IPython.display import display

In [2]:
protein_cache = {}

In [3]:
def get_pdb(identifier):
    directory = './PDBfiles/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path for the PDB file
    pdb_file = os.path.join(directory, f"{identifier}.pdb")

    # Check if the PDB file already exists
    if not os.path.exists(pdb_file):
        # Fetch the PDB file from the RCSB PDB database
        pdbl = PDBList()
        pdbl.retrieve_pdb_file(identifier, pdir=directory, file_format='pdb')
        print(f"PDB file '{pdb_file}' not found.")
        print(f"Current directory contents: {os.listdir(directory)}")
        
        # Rename the file to match the identifier
        fetched_file = os.path.join(directory, f"pdb{identifier.lower()}.ent")
        if os.path.exists(fetched_file):
            os.rename(fetched_file, pdb_file)
    
    try:
        with open(pdb_file, 'r') as f:
            pdb_content = f.read()
    except Exception as e:
        print(f"Error reading PDB file '{pdb_file}': {e}")
        return None
    
    return pdb_content

def get_chain_identifiers(pdb_content):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', StringIO(pdb_content))
    chains = set()
    for model in structure:
        for chain in model:
            chains.add(chain.id)
    print()
    return sorted(list(chains))

In [4]:
def update_visibility_protein(style):
    if style == 'cartoon':
        radius_protein.layout.visibility = 'hidden'
        scale_protein.layout.visibility = 'hidden'
    elif style =='stick':
        radius_protein.layout.visibility = 'visible'
        scale_protein.layout.visibility = 'hidden'
    elif style =='sphere':
        radius_protein.layout.visibility = 'hidden'
        scale_protein.layout.visibility = 'visible'

def chain_styles_dict(selected_chains, chain_style, chain_color, chain_radius, chain_scale):
    """Create a dictionary of chain styles based on selected chains and their styles."""
    styles = {}
    for chain in selected_chains:
        styles[chain] = {
            'style': chain_style,
            'color': chain_color,
            'radius': chain_radius,
            'scale': chain_scale
        }
    return styles

In [5]:
identifier_widget = Text(
    value='1ZNI',  
    placeholder='Enter identifier...',
    description='Identifier:',
    disabled=False
)

style_protein = Dropdown(
    options=['cartoon', 'stick', 'sphere',],
    value='cartoon',
    description='Style:')

color_protein = Dropdown(
    options=['spectrum', 'chain', 'element'],
    value='spectrum',
    description='Color:'
)

radius_protein = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Atomic radius size') 

scale_protein = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Sphere size')

asa_checkbox = Checkbox(
    value = False,
    description = 'Add surface'
)

asa_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='ASA surface opacity')

chain_select = Dropdown(
    options=[],  
    value=None,
    description='Chains:'
)

chain_style = Dropdown(
    options=['cartoon', 'stick', 'sphere'],
    value='cartoon',
    description='Chain Style:'
)

chain_color = Dropdown(
    options=['spectrum', 'chain', 'element'],
    value='spectrum',
    description='Chain Color:'
)

chain_radius = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Chain Atomic radius size'
) 

chain_scale = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Chain Sphere size'
)

In [6]:
def add_protein_model(view, pdb_content, global_style, global_color, global_radius, global_scale, chain_styles, chain_select, surface, opacity):
    """Add the PDB model to the 3Dmol view and apply the given style."""
    view.addModel(pdb_content, 'pdb')
    view.setBackgroundColor('#000000')
    
    # Apply global styles
    if global_style == 'cartoon':
        view.setStyle({'cartoon': {'color': global_color}})
    elif global_style == 'stick':
        view.setStyle({'stick': {'colorscheme': global_color, 'radius': global_radius}})
    elif global_style == 'sphere':
        view.setStyle({'sphere': {'colorscheme': global_color, 'scale': global_scale}})
    
    # Apply chain-specific styles
    for chain, style in chain_styles.items():
        if style['style'] == 'cartoon':
            view.setStyle({'chain': chain, 'cartoon': {'color': style['color']}})
        elif style['style'] == 'stick':
            view.setStyle({'chain': chain, 'stick': {'colorscheme': style['color'], 'radius': style['radius']}})
        elif style['style'] == 'sphere':
            view.setStyle({'chain': chain, 'sphere': {'colorscheme': style['color'], 'scale': style['scale']}})
    
    if surface:
        view.addSurface(py3Dmol.VDW, {'opacity': opacity, 'color': 'spectrum'})


In [7]:
def viewProtein(identifier, global_style='cartoon', global_color='spectrum', global_radius=0.2, global_scale=1, chain_styles={}, chain_select=None, surface=False, opacity=1):
    """Visualize a molecule in 3D"""
    
    if identifier not in protein_cache:
        # Fetch the PDB file for the new protein identifier
        pdb_content = get_pdb(identifier)
        if not pdb_content:
            print("Failed to fetch PDB file.")
            return None
        # Cache the loaded PDB file
        protein_cache[identifier] = pdb_content
    else:
        pdb_content = protein_cache[identifier]
    
    view = py3Dmol.view(width=800, height=800)
    add_protein_model(view, pdb_content, global_style, global_color, global_radius, global_scale, chain_styles, chain_select, surface, opacity)
    view.zoomTo()
    return view

def update_chain_select(identifier):
    pdb_content = get_pdb(identifier)
    if not pdb_content:
        return None
    chains = get_chain_identifiers(pdb_content)
    chain_select.options = chains
    chain_select.value = chains[0]

def update_protein_viewer(identifier, global_style, global_color, global_radius, global_scale, chain_styles, chain_select, surface, opacity):
    view = viewProtein(
        identifier=identifier,
        global_style=global_style,
        global_color=global_color,
        global_radius=global_radius,
        global_scale=global_scale,
        chain_styles=chain_styles,
        chain_select=chain_select,
        surface=surface,
        opacity=opacity
    )
    display(view)

interact(update_protein_viewer,
         identifier=identifier_widget,
         global_style=style_protein,
         global_color=color_protein,
         global_radius=radius_protein,
         global_scale=scale_protein,
         chain_styles=fixed({}),
         chain_select=chain_select,
         surface=asa_checkbox,
         opacity=asa_slider)

style_protein.observe(lambda change: update_visibility_protein(change['new']), names='value')
#update_chain_select('1ZNI')

interactive(children=(Text(value='1ZNI', description='Identifier:', placeholder='Enter identifier...'), Dropdo…